In [1]:
%run ../../_setup.py

from netCDF4 import Dataset
import xarray

import numpy as np

import pygmt
# from gmt import Figure


import matplotlib.pyplot as plt
%matplotlib notebook

from OceanPy.readwrite import readxyz

from scipy.interpolate import griddata
import cmocean as cmo

# Bathymetry (Smith and Sandwell, 1997)

In [2]:
# input_file = os.path.join(datadir, 'external', 'bathymetry', 'smith_and_sandwell_1997', 'bathymetry_-48_137_-58_157.nc')
# input_file = os.path.join(datadir, 'external', 'bathymetry', 'bodc_res30sec', 'GEBCO_2014_2D_137.0_-58.0_147.0_-48.0.nc')
input_file = os.path.join(datadir, 'external', 'bathymetry', 'gebco_2019', 'gebco_2019_n-45.0_s-60.0_w135.0_e160.0.nc')

bathy = xarray.open_dataset(input_file)

# boundaries
north = -52.4
west = 137
south = -48
east = 147

# dep = bathy.depth.sel(longitude=slice(west, east), latitude=slice(north, south))
dep = bathy.elevation.sel(lon=slice(west, east), lat=slice(north, south))
depths = np.linspace(-5000, 0, 11)


# x, y, z = readxyz(os.path.join(datadir, 'external', 'bathymetry', 'smith_and_sandwell_1997_137_-52.4_147_-48.xyz'))

# # grid  1 minute
# loni = np.linspace(137, 147, 60*(147-137)+1)
# lati = np.linspace(-48, -52.4, int(60*(-48 + 52.4)+1))

# lonlon, latlat = np.meshgrid(loni, lati)
# elev = griddata((x, y), z, (lonlon, latlat), method='linear')

# fig, ax = plt.subplots(2, 1)
# ax[0].scatter(x, y, c=z)
# ax[1].pcolor(lonlon, latlat, np.ma.masked_invalid(elev))


# Sea surface heights and derived variables
http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047


TODO: open different ssh rep file and make selection of data from 136.5, such that ssh contours cover the whole figure

In [3]:
input_file = os.path.join(datadir, 'external', 'ssh', 'rep', 'SEALEVEL_GLO_PHY_L4_REP_OBS_137_-52.4_147_-48_199801-199812.nc')

# Select data over time of the voyage
ssh = xarray.open_dataset(input_file)
SSHvoyage = ssh.sel(time=slice('1998-03-10', '1998-03-28'))

# Mean Absolute Dynamic Topography (ADT) over time of voyage
voyagemeanadt = SSHvoyage.mean('time')['adt']

# Weekly mean ADT
weeks = np.unique(SSHvoyage.time.dt.week)
weekmeanadt = ssh.groupby('time.week').mean('time')['adt'][weeks-1]


# CTD stations SS9802 and SR3 stations AA9802

In [5]:
input_file = os.path.join(datadir, 'ss9802', 'processed', 'ctd', 'ss9802_ctd.nc')
ctd = Dataset(input_file)

input_file = os.path.join(datadir, 'au9706', 'processed', 'au9706_ctd.nc')
sr3 = Dataset(input_file)

In [6]:
stations = [2, 18, 19, 21, 23, 40, 41, 44, 46, 67, 66, 65, 53, 73]
istations = [i for i, station in enumerate(sr3['station'][:]) if station in stations]


In [7]:

# from netCDF4 import num2date
# # help(num2date)
# from gsw import SA_from_SP, CT_from_t, z_from_p, grav, geo_strf_dyn_height

In [8]:
# lon = sr3['lon'][:][:, np.newaxis]
# lat = sr3['lat'][:][:, np.newaxis]

# time = num2date(sr3['time'][:], sr3['time'].units)

# nplevels = sr3['Pressure_row_size'][:]
# p_ref = 1500
# # nstations, npmax = len(stations), max(nplevels)

# t = np.ma.masked_all((len(nplevels), max(nplevels)))
# SP, p, z = t.copy(), t.copy(), t.copy()
# for ist, ip in enumerate(nplevels):
#     i = np.sum(nplevels[:ist])
#     ii = np.sum(nplevels[:ist+1])

#     t[ist, :ip] = sr3['Temperature'][i:ii]
#     SP[ist, :ip] = sr3['Salinity'][i:ii]
#     p[ist, :ip] = sr3['Pressure'][i:ii]
#     z[ist, :ip] = sr3['z'][i:ii]

# SA = SA_from_SP(SP, p, lon, lat)
# CT = CT_from_t(SA, t, p)
# deltaD = np.ma.masked_invalid(geo_strf_dyn_height(SA.data, CT.data, p, p_ref=p_ref, axis=1))
# g = grav(lat, p)

# sr3_gsw = {}


In [9]:
# # # find indices for pressure levels in CTD prodiles
# # pressure_levels = {}
# # for ip, p in enumerate(ctd['p'][:]):
# #     pressure_levels[p] = ip

# # # define reference and interest level 
# # # p_ref = 1494
# # p_int = 2

# # # get dynamic height contours
# # D = np.array([ctd['deltaD'][profile, pressure_levels[p_int]] / 
# #               ctd['g'][profile, pressure_levels[p_int]]
# #               for profile in range(len(ctd.dimensions['profile']))])


# # find indices for pressure levels in CTD prodiles
# pressure_levels = {}
# for ip, pres in enumerate(p[1].data):
#     pressure_levels[pres] = ip

# # define reference and interest level 
# # p_ref = 1494
# p_int = 2

# # get dynamic height contours
# D = np.array([deltaD[profile, pressure_levels[p_int]] / 
#               g[profile, pressure_levels[p_int]]
#               for profile in range(len(nplevels))])

In [10]:
# fig, ax = plt.subplots(figsize=(12,8))
# ax.scatter(ctd['lon'][2:,0], ctd['lat'][2:,0], facecolors='r')
# ax.plot(sr3['longitude'][istations], sr3['latitude'][istations], 'k.-')
# ax.scatter(sr3['longitude'][:], sr3['latitude'][:], facecolors='g')

# # plt.scatter(sr3['lon'][stations], sr3['lat'][stations], facecolors='y')
# ax.axis('equal')


# Orsi et al. 1995 fronts

In [11]:
orsi = {}
fronts = {'stf': 'tomato', 'saf': 'skyblue', 'pf': 'royalblue'}
for front in fronts.keys():
    input_file = os.path.join(datadir, 'external', 'orsi', front + '.txt')
    lon, lat = zip(*np.genfromtxt(input_file, delimiter='   ', comments='%'))
    orsi[front] = [lon, lat]


In [12]:
cmap = pygmt.makecpt(C='jet', T=[-5000, -1000])
print(cmap)

None


In [13]:
import matplotlib.colors as mcolors
def gmtColormap(cptf, name=None):
    """Read a GMT color map from an OPEN cpt file
    Parameters
    ----------
    cptf : open file or url handle
        path to .cpt file
    name : str, optional
        name for color map
        if not provided, the file name will be used
    """
    # generate cmap name
    if name is None:
#         name = '_'.join(os.path.basename(cptf.name).split('.')[:-1])
        filename = os.path.basename(os.path.join(cptf))
        name = os.path.splitext(filename)[0]
    
    f = open(cptf)
    
    # process file
    x = []
    r = []
    g = []
    b = []
    lastls = None
    for l in f.readlines():
        ls = l.split()

        # skip empty lines
        if not ls:
            continue

        # parse header info
        if ls[0] in ["#", b"#"]:
            if ls[-1] in ["HSV", b"HSV"]:
                colorModel = "HSV"
            else:
                colorModel = "RGB"
            continue

        # skip BFN info
        if ls[0] in ["B", b"B", "F", b"F", "N", b"N"]:
            continue

        # parse color vectors
        x.append(float(ls[0]))
        r.append(float(ls[1]))
        g.append(float(ls[2]))
        b.append(float(ls[3]))

        # save last row
        lastls = ls

    x.append(float(lastls[4]))
    r.append(float(lastls[5]))
    g.append(float(lastls[6]))
    b.append(float(lastls[7]))
    
    x = np.array(x)
    r = np.array(r)
    g = np.array(g)
    b = np.array(b)

    if colorModel == "HSV":
        for i in range(r.shape[0]):
            # convert HSV to RGB
            rr,gg,bb = colorsys.hsv_to_rgb(r[i]/360., g[i], b[i])
            r[i] = rr ; g[i] = gg ; b[i] = bb
    elif colorModel == "RGB":
        r /= 255.
        g /= 255.
        b /= 255.

    red = []
    blue = []
    green = []
    xNorm = (x - x[0])/(x[-1] - x[0])
    for i in range(len(x)):
        red.append([xNorm[i],r[i],r[i]])
        green.append([xNorm[i],g[i],g[i]])
        blue.append([xNorm[i],b[i],b[i]])

    # return colormap
    cdict = dict(red=red,green=green,blue=blue)
    return mcolors.LinearSegmentedColormap(name=name,segmentdata=cdict)

def export_cmap_to_cpt(cmap, vmin=0, vmax=1, N=255, filename="test.cpt",**kwargs):
    # create string for upper, lower colors
    b = np.array(kwargs.get("B", cmap(0.)))
    f = np.array(kwargs.get("F", cmap(1.)))
    na = np.array(kwargs.get("N", (0,0,0))).astype(float)
    ext = (np.c_[b[:3],f[:3],na[:3]].T * cmap.N).astype(int)
    extstr = "B {:3d} {:3d} {:3d}\nF {:3d} {:3d} {:3d}\nN {:3d} {:3d} {:3d}"
    ex = extstr.format(*list(ext.flatten()))
    #create colormap
    cols = (cmap(np.linspace(0.,1.,N))[:,:3]*cmap.N).astype(int)
    vals = np.linspace(vmin, vmax, N)
    arr = np.c_[vals[:-1],cols[:-1],vals[1:],cols[1:]]
    # save to file
    fmt = "%e %3d %3d %3d %e %3d %3d %3d"
    np.savetxt(filename, arr, fmt=fmt, 
               header="# COLOR_MODEL = RGB",
               footer = ex, comments="")


In [14]:
from pygmt import which
which.aliases

{'G': 'download'}

In [24]:
Nin = 256
Nout = 6
    
# test case: create cpt file from cmocean colormap
cmap = plt.get_cmap(cmo.cm.deep_r, cmo.cm.deep_r.N)
# cmap = gmtColormap_openfile('cmaps/GMT_seafloor.cpt')
# Nin = cmap.N

# you may create your colormap differently, as in the question
export_cmap_to_cpt(cmap, vmin=-5000, vmax=-1000, N=Nout, filename="./cmaps/cmocean_deep.cpt")

In [25]:
odds = [1, 2, 9, 10, 11, 12, 27, 45, 46, 47, 75, 76, 77, 78, 101]
iodds = [istat for istat, station in enumerate(ctd['station']) if station in odds]

In [2]:
fig = pygmt.Figure()
fig.coast(region=[137, 147, -52.4, -48], projection='B141/-51/-50/-49/5i', land='black', water='skyblue', frame=True)
# fig.contour(lnln.flatten(), ltlt.flatten(), dep.data.flatten(), pen='1p,black')

# -DJRM+o0.6i/0+mc -I0.3 -Bx2+lTOPO -By+lkm
# plot bathymetry
fig.grdimage(dep, projection='B141/-51/-50/-49/5i', region=[137, 147, -52.4, -48], cmap='./cmaps/cmocean_deep.cpt')#, shading='+a120+nt1.5')
fig.colorbar(
    cmap="./cmaps/cmocean_deep.cpt",
    position="n0.1/-0.2+w10/0.5+h",  # plot using paper coordinates (x) at 1.2cm,4.75cm
    frame=["xaf", "y+lDepth\t(m)"], #-DJRM+o0.6i/0+e+mc
    scale=1,
)
# plot SSH contours
for i in range(weekmeanadt.shape[0]):
    fig.grdcontour(weekmeanadt[i], interval=0.2, annotation='-', limit=[-0.2, 0.4], pen='gray')
fig.grdcontour(voyagemeanadt, interval=0.2, annotation=0.2, label_placement='l144/52S/145/48S', limit=[-0.2, 0.4], pen='1p,gray')

# plot AU9706 stations and shiptrack
fig.plot(x=sr3['longitude'][istations], y=sr3['latitude'][istations], pen='0.7p,gold')
fig.plot(x=sr3['longitude'][istations], y=sr3['latitude'][istations], style='s0.3c', color='gold', pen='black')

# plot SS9802 stations and shiptrack
fig.plot(x=ctd['lon'][2:,0], y=ctd['lat'][2:,0], pen='1p,white')
fig.plot(x=ctd['lon'][2:,0], y=ctd['lat'][2:,0], style='c0.2c', color='white', pen='black')
fig.plot(x=ctd['lon'][iodds,0], y=ctd['lat'][iodds,0], style='c0.2c', color='orange', pen='black')
# fig.plot(x=ctd['lon'][2:,0]+0.01, y=ctd['lat'][2:,0]+0.1, text='1', style=)

# plot globe subplot and mean SSH contours of fronts from Orsi et al. 1995
fig.coast(region="g", projection="G141/-50.1/10000/0/0/0/0/0/2i", land="black", water='white', frame='afg', X='3.8i', Y='2.2i')
for front in fronts.keys():
    fig.plot(orsi[front][0], orsi[front][1], pen='1p,gray') #'0.75p,%s' %fronts[front]
#     fig.plot([140, 150], [-50, -52], text='5p'+'hello')

# plot SS9802 survey area in subplot
fig.plot([137.0, 147.0, 147.0, 137.0, 137.0], [-52.4, -52.4, -48.0, -48.0, -52.4], pen='0.5p,red')
fig.plot(x=sr3['longitude'][istations], y=sr3['latitude'][istations], pen='0.1p,gold')
fig.plot(x=sr3['longitude'][istations], y=sr3['latitude'][istations], style='s0.075c', color='gold', pen='gold')

fig.text(x=105, y=-53, angle=25,text='PF',font='7p,black')
fig.text(x=105, y=-48, angle=25,text='SAF',font='7p,black')
fig.text(x=105, y=-41, angle=25,text='STF',font='7p,bold,black')


# colorbar pygmt.makecpt
fig.show(dpi=300, width=1000)

NameError: name 'pygmt' is not defined

In [27]:
extensions = ['.png', '.pdf', '.eps']
for extension in extensions:
    fig.savefig(os.path.join(figsdir, 'manuscripts', '01_manuscript', '01_overview-map' + extension), transparent=True)

GMTInvalidInput: Transparency unavailable for 'pdf', only for png.

In [36]:
plt.rc('text', usetex=True)

In [35]:
pygmt.mathops

<module 'pygmt.mathops' from '/home/janjaapmeijer/miniconda3/envs/ocean36/lib/python3.6/site-packages/pygmt/mathops.py'>

In [ ]:
fs_small=12
fs_large = 14
plt.rc('xtick',labelsize=fs_small)
plt.rc('ytick',labelsize=fs_small)
plt.rc('text', usetex=True)